## IMPORTS

In [1]:
import os
import cv2

import numpy as np
from numpy.random import seed
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import regex as re

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.random import set_seed

# 1-Off Accuracy

## Loading Inputs

In [2]:
inputs_path = "../data/cnn_inputs/"

In [3]:
y_test_cat = np.load(inputs_path+"y_test_cat.npy")
y_pred = np.load(inputs_path+"y_pred.npy")

In [4]:
print(len(y_pred),"....",type(y_pred))
print(len(y_test_cat),"....",type(y_test_cat))

5502 .... <class 'numpy.ndarray'>
5502 .... <class 'numpy.ndarray'>


In [5]:
def one_off_accuracy(y_test_cat,y_pred):
    
    i_ind = []
    accurate = 0
    
    hit_miss = pd.DataFrame(data = 0, dtype="int16"
                          ,index = np.linspace(0,y_test_cat.shape[1]-1,y_test_cat.shape[1],dtype="int8")
                          ,columns = ['Hit','Miss'])
    
    conf = pd.DataFrame(data = 0, dtype="int16"
                          ,index = np.linspace(0,y_test_cat.shape[1]-1,y_test_cat.shape[1],dtype="int8")
                          ,columns = np.linspace(0,y_test_cat.shape[1]-1,y_test_cat.shape[1],dtype="int8"))
    
    for i in range(len(y_test_cat)):
        
        # 1-Off Accuracy
        i_ind = [y_pred[i].argmax()-1,y_pred[i].argmax(),y_pred[i].argmax()+1]
        if y_test_cat[i].argmax() in i_ind:
            accurate += 1
    
        # Hit/Miss Matrix
            hit_miss.at[y_test_cat[i].argmax(),"Hit"] += 1
        else:
            hit_miss.at[y_test_cat[i].argmax(),"Miss"] += 1
            
        # Confusion Matrix
        conf.at[y_test_cat[i].argmax(),y_pred[i].argmax()] += 1
    
    conf["Total"] = np.sum(y_test_cat,axis=0,dtype="int16")
    for k in range(0,y_test_cat.shape[1]):
        if (conf.iloc[k,k] !=0) and (conf.at[k,"Total"]!=0):
            conf.at[k,"Hit Rate"] = round(100*np.divide(conf.at[k,k],conf.at[k,"Total"]),1)
            
    print(f"The 1-Off Accuracy is {round(100*accurate/len(y_test_cat),2)}%")
    return hit_miss,conf

In [9]:
hit_miss,conf = one_off_accuracy(y_test_cat,y_pred)

The 1-Off Accuracy is 59.49%


In [7]:
conf

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,Total,Hit Rate
0,902,18,6,6,0,0,274,0,0,0,...,7,16,19,0,0,0,0,0,1248,72.3
1,222,50,43,26,0,0,57,0,0,0,...,6,15,19,0,0,0,0,0,438,11.4
2,17,9,46,19,0,1,26,0,0,0,...,0,1,2,0,0,0,0,0,121,38.0
3,29,6,35,42,0,3,50,0,0,0,...,1,0,2,0,0,0,0,0,168,25.0
4,27,0,2,24,1,5,128,0,0,0,...,0,0,0,0,0,0,0,0,187,0.5
5,55,0,0,7,0,15,408,0,0,0,...,0,0,1,0,0,0,0,0,486,3.1
6,239,0,1,7,0,18,773,0,0,0,...,0,3,3,0,0,0,0,0,1044,74.0
7,230,0,1,0,0,0,306,0,0,0,...,3,3,2,0,0,0,0,0,545,NaN
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [10]:
hit_miss

,Hit,Miss
0,920,328
1,315,123
2,74,47
3,77,91
4,30,157
5,423,63
6,791,253
7,306,239
8,0,0
9,0,0
